In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util


model = SentenceTransformer('intfloat/e5-small-v2')
input_texts = [
    'query: how much protein should a female eat',
    'query: summit define',
    "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    "passage: Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."
]



In [18]:
embeddings = model.encode(input_texts, normalize_embeddings=True)


In [2]:
import os, re


In [3]:
os.getcwd()

'/Users/kacperjarzyna/systemy_rozproszone/test_notebooks'

In [61]:
data.shape

(48, 4)

In [76]:
quantites = np.random.randint(1,100,size=48)
prices_buy = np.random.randint(1,2,size=48)*10 + np.random.randint(0,10,size=48) + np.random.randint(0,10,size=48)*0.1
prices_sale = prices_buy - np.random.randint(0,10,size=48) + np.random.randint(0,10,size=48)*0.1


In [100]:
data = pd.read_csv('data/flowershopdata.csv')
data['Description'] = data[['Description', 'Needs','Season']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
data['Buy_price'] = prices_buy
data['Sale_price'] = np.round(prices_sale,2)
data['Quantities'] = quantites
jpg_paths = [f"/Users/kacperjarzyna/systemy_rozproszone/test_notebooks/data/img_flowers/{flower.lower()}.jpg" for flower in data['Name']]
data['pic_path'] = jpg_paths
data.drop(columns=['Needs','Season'],inplace=True)

In [101]:
data.to_csv('data/flowershopdata_clean.csv')

In [46]:
import os
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def fetch_first_image_url(query):
    options = webdriver.SafariOptions()
    driver = webdriver.Safari(options=options)

    search_url = f"https://pl.pinterest.com/search/pins/?q={query}%20flower&rs=typed"
    driver.get(search_url)

    time.sleep(2)
    image_elements = driver.find_elements(By.CSS_SELECTOR, 'img')

    if len(image_elements) >= 10:
        # Get the URL of the 10th image
        image_element = image_elements[9]  # Index 9 corresponds to the 10th image (0-based index)
        image_url = image_element.get_attribute('src')
    else:
        print("Not enough images found on the page")
        image_url = None

    driver.quit()
    return image_url

def save_image_from_url(url, directory, filename):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    image_path = os.path.join(directory, filename)
    image.save(image_path)
    print(f"Image saved to {image_path}")



In [82]:
import nanoid
def create_product(product_data):
    base_url = "http://127.0.0.1:8000"
    response = requests.post(f"{base_url}/products/", json=product_data)
    print(response.json())

In [109]:
for index,item in data.iterrows():
    product_id = nanoid.generate(size=10)
    product_data = {
    "id": product_id,
    "name": item['Name'][0] + item['Name'][1:].lower(),
    "description": item['Description'],
    "sale_price": item['Sale_price'],
    "quantity": item['Quantities'],
    "buy_price": item['Buy_price'],
    "date": 'None',
    "picture_path": item['pic_path']
    }
    create_product(product_data)


{'id': 'CLsWtAJvtF', 'name': 'Periwinkle', 'description': 'Perfect as a ground cover, the flowers resemble small, delicate stars and are often found in shades of blue, purple, and white. Needs droughty. Blooms in mid-spring to mid-fall.', 'sale_price': 7.0, 'quantity': 2, 'buy_price': 12.3, 'date': 'None', 'picture_path': '/Users/kacperjarzyna/systemy_rozproszone/test_notebooks/data/img_flowers/periwinkle.jpg'}
{'id': 'RXqJydTV5C', 'name': 'Bellflower', 'description': 'Known for their resemblance to bells, these flowers come in shades of blue, purple, pink, and white and add a whimsical touch to any garden. Needs well-drained soil. Blooms in early summer to early fall.', 'sale_price': 4.3, 'quantity': 36, 'buy_price': 12.8, 'date': 'None', 'picture_path': '/Users/kacperjarzyna/systemy_rozproszone/test_notebooks/data/img_flowers/bellflower.jpg'}
{'id': 'FiH8MNR-Eh', 'name': 'Lily', 'description': 'A highly recognized flower which has bright colors and a distinct fragrance, lilies are of

In [85]:
data.items()

TypeError: 'generator' object is not subscriptable

In [108]:
import pymongo

# Connect to the MongoDB server
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Select your database
db = client["ecommerce-app"]

# List all collection names in the database
collection_names = db.list_collection_names()


collection = db['products']
# Find all documents in the collection
documents = collection.find()
for x in documents:
    print(x)



{'_id': ObjectId('66588990e11c5520466d10ee'), 'id': 'oMRvye87EN', 'name': 'dwa', 'description': 'dwa', 'sale_price': 0.01, 'quantity': 1, 'buy_price': 0.01, 'date': '2024-05-30', 'picture_path': '66588990e11c5520466d10ec'}


In [107]:
db.drop_collection('products')
for document in documents:
    print(document)